In [1]:
#importing dependencies and dataset
import numpy as np
import pandas as pd
from pathlib import Path
import tarfile
import urllib.request
from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error
from sklearn.cluster import KMeans
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics.pairwise import rbf_kernel

 

def housing_dataset():
    tar_path = Path('datasets/housing.tgz')
    if not tar_path.is_file():
        Path('datasets').mkdir(exist_ok=True,parents=True)
        url = urllib.request.urlretrieve('https://github.com/ageron/data/raw/main/housing.tgz',tar_path)
        with tarfile.open(tar_path) as housing_file:
            housing_file.extractall(path='datasets',filter='data')
    return pd.read_csv(Path("datasets/housing/housing.csv"))

housing = housing_dataset()
housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [2]:
#creating a new colume with income categories
housing['income_cat'] = pd.cut(
    
    housing['median_income'],
    bins=[0,1.5,3,4.5,6,np.inf],
    labels=[1,2,3,4,5]
    
)

housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity,income_cat
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY,5
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY,5
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY,5
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY,4
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY,3


In [3]:
strat_train_set,strat_test_set = train_test_split(
    housing,
    test_size=0.2,
    stratify=housing['income_cat'],
    random_state=42
)

In [4]:
for set_ in (strat_train_set,strat_test_set):
    set_.drop('income_cat',axis =1,inplace=True)

In [5]:
housing = strat_train_set.drop('median_house_value',axis= 1)
housing_labels = strat_train_set['median_house_value'].copy()

In [6]:
#Have created a custom Transformer
class ClusterSimilarity(BaseEstimator,TransformerMixin):
    def __init__(self,n_clusters = 10,gamma = 1.0,random_state = None):
        self.n_clusters = n_clusters
        self.gamma = gamma
        self.random_state = random_state
    
    def fit(self,X,y=None,sample_weight=None):
        self.kmeans_ = KMeans(
            n_clusters=self.n_clusters,
            random_state=self.random_state
        )
        self.kmeans_.fit(X,sample_weight = sample_weight)
        return self
    
    def transform(self,X):
        return rbf_kernel(
            X,
            self.kmeans_.cluster_centers_,
            gamma=self.gamma
        )
        
    def get_feature_names_out(self,input_features=None):
        return [f"cluster_{i}_sim" for i in range(self.n_clusters)]

In [7]:
#ratio pipeline

def column_ratio(X):
    return X[:,[0]]/X[:,[1]]

def ratio_pipeline():
    return make_pipeline(
        SimpleImputer(strategy='median'),
        FunctionTransformer(column_ratio),
        StandardScaler()
    )


In [8]:
#data_preprocessing

log_pipeline = make_pipeline(
    SimpleImputer(strategy='median'),
    FunctionTransformer(np.log,feature_names_out='one-to-one'),
    StandardScaler()
)

default_num_pipeline = make_pipeline(
    SimpleImputer(strategy='median'),
    StandardScaler()
)


cat_pipeline = make_pipeline(
    OneHotEncoder(handle_unknown='ignore'),
    SimpleImputer(strategy='most_frequent')
)

cluster_simil = ClusterSimilarity(
    n_clusters=10,
    gamma=1.0,
    random_state=42
)

preprocessing = ColumnTransformer(
    transformers=[
        ('bedrooms_ratio',ratio_pipeline(),['total_bedrooms','total_rooms']),
        ('rooms_per_house',ratio_pipeline(),['total_rooms','households']),
        ('people_per_house',ratio_pipeline(),['population','households']),
        ('log',log_pipeline,['total_bedrooms','total_rooms','population','households','median_income']),
        ('geo',cluster_simil,['latitude','longitude']),
        ('cat',cat_pipeline,make_column_selector(dtype_include=object))
    ],
    remainder=default_num_pipeline
)

In [9]:
#model 
lin_reg = Pipeline([
    ('preprocessing',preprocessing),
    ('model',LinearRegression())
])

In [10]:
#fit model
lin_reg.fit(housing, housing_labels)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocessing', ...), ('model', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('bedrooms_ratio', ...), ('rooms_per_house', ...), ...]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",Pipeline(step...ardScaler())])
,"sparse_threshold sparse_threshold: float, defa

In [11]:
housing_predictions = lin_reg.predict(housing)
housing_predictions[:5].round(-2), housing_labels.iloc[:5].values

(array([246000., 372700., 135700.,  91400., 330900.]),
 array([458300., 483800., 101700.,  96100., 361800.]))

In [12]:
lin_rmse = root_mean_squared_error(housing_labels, housing_predictions)
lin_rmse

68972.88910758485